In [4]:
from src.data_utils.ingestion import BronzeIngestor
from src.data_utils.processor import SilverProcessor
from src.data_utils.builder import GoldBuilder

In [2]:
db_path = "data/processed/lakehouse.sqlite"
csv_path = "data/raw/characteristics_raw.csv"

# Bronze Files

In [ ]:
ingestor = BronzeIngestor(db_path, csv_path)
ingestor.run()

Ingesting data/raw/characteristics_raw.csv in chunks...
Processed chunk 1...
Processed chunk 6...
Processed chunk 11...
Processed chunk 16...
Processed chunk 21...
Processed chunk 26...
Processed chunk 31...
Processed chunk 36...
Processed chunk 41...
GKX characteristics loaded into 'bronze_characteristics'.
Starting CRSP download: 1957-01-31 to 2021-12-31...
CRSP data loaded into 'bronze_crsp'.


# Silver Files

In [3]:
processor = SilverProcessor(db_path)
processor.run()

Starting Silver processing (Rank Standardization)...
Processed date 19570131 (0/780 = 0%)
Processed date 19610330 (50/780 = 6%)
Processed date 19650528 (100/780 = 13%)
Processed date 19690731 (150/780 = 19%)
Processed date 19730928 (200/780 = 26%)
Processed date 19771130 (250/780 = 32%)
Processed date 19820129 (300/780 = 38%)
Processed date 19860331 (350/780 = 45%)
Processed date 19900531 (400/780 = 51%)
Processed date 19940729 (450/780 = 58%)
Processed date 19980930 (500/780 = 64%)
Processed date 20021129 (550/780 = 71%)
Processed date 20070131 (600/780 = 77%)
Processed date 20110331 (650/780 = 83%)
Processed date 20150529 (700/780 = 90%)
Processed date 20190731 (750/780 = 96%)
Silver processing complete.
Running diagnostics...
Range check passed: [-0.4999, 0.4999]
Info: 2365 stocks have < 12 months of data (out of 32793 total).


# Gold Files

In [7]:
builder = GoldBuilder(db_path)
builder.run()

Building Gold Panel (Joining Returns)...
Creating temporary indexes...
Gold panel built successfully.
Sample of Gold Panel (Check dates manually):
              date_fmt  target_ret_excess
0  1957-01-31 00:00:00           0.016105
1  1957-01-31 00:00:00          -0.002300
2  1957-01-31 00:00:00          -0.006442
3  1957-01-31 00:00:00           0.076080
4  1957-01-31 00:00:00          -0.020031
